In [12]:
#Import libraries
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
import time
import re

In [13]:
title = " Analista de datos"     #   Job title
location = "Mexico"             #   Job location
start = 0                       #   Starting point for pagination

In [14]:
#   URL for LinkedIn job search
list_url = f"https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords={title}&location={location}&start={start}"

#   Send a GET request to the URL and store the response
response = requests.get(list_url)

#   Get the HTML, parse the response and find all list items(jobs postings)
list_data = response.text
list_soup = BeautifulSoup(list_data, "html.parser")
page_jobs = list_soup.find_all("li")

In [15]:
#   Check if the HTML contains the job postings
response.text

'<!DOCTYPE html>\n\n      <li>\n        \n    \n\n    \n    \n    \n      <div class="base-card relative w-full hover:no-underline focus:no-underline base-card--link base-search-card base-search-card--link job-search-card" data-entity-urn="urn:li:jobPosting:4128247651" data-impression-id="jobs-search-result-0" data-reference-id="YHTG2bHV7Tx/cmSaB2R0GA==" data-tracking-id="/zLMdCABf64GJLxMG2LsmA==" data-column="1" data-row="1">\n        \n\n        <a class="base-card__full-link absolute top-0 right-0 bottom-0 left-0 p-0 z-[2]" href="https://mx.linkedin.com/jobs/view/analista-de-datos-jr-at-walmart-de-m%C3%A9xico-y-centroam%C3%A9rica-4128247651?position=1&amp;pageNum=0&amp;refId=YHTG2bHV7Tx%2FcmSaB2R0GA%3D%3D&amp;trackingId=%2FzLMdCABf64GJLxMG2LsmA%3D%3D" data-tracking-control-name="public_jobs_jserp-result_search-card" data-tracking-client-ingraph data-tracking-will-navigate>\n          \n          <span class="sr-only">\n              \n        \n        Analista de datos Jr\n      \n

In [16]:
#   Create an empty list to store the job postings
id_list = []

In [17]:
#   Itetrate through job postings to find job ids
for job in page_jobs:
    base_card_div = job.find("div", {"class": "base-card"})
    job_id = base_card_div.get("data-entity-urn").split(":")[3]
    print(job_id)
    id_list.append(job_id)

4128247651
4137031981
4114591465
4084883735
4126320994
4122468515
4137904013
4132496530
4119058631
4079130909


In [18]:
# Initialize an empty list to store job information
job_list = []

# Lista de palabras clave para habilidades técnicas
technical_skills_keywords = [
    "Statistics", "Data Analysis", "Programming Languages", "Python", "Contemporary Database Systems",
    "Marketing Analytics", "Experiment Design", "Consumer Behavior Analysis", "Optimization",
    "Solutions", "Object", "Experience in data analysis", "Data analysis tools", "Data visualization software",
    "Complex data models", "Query optimization", "Google Sheets", "Microsoft Excel", "CSV files",
    "Sales information manipulation", "Report generation", "Trend analysis", "Statistical analysis",
    "Problem-solving", "Data collection", "Data processing", "Data cleaning", "Data processing and reporting",
    "Data visualization tools", "Data connection", "Query development", "Data reconciliation",
    "eCommerce environment", "Cloud computing platforms", "AWS", "Google Cloud", "Azure",
    "ETL processes design", "Process administration", "Advanced Excel", "VBA macros", "SQL database",
    "Office suite", "SAP", "SAP macros", "Artificial Intelligence", "Back-end programming languages",
    "Front-end programming languages", "Big Data", "Data Science", "IT Administration", "Data pipelines",
    "Predictive models", "Machine Learning", "Data interpretation", "Deep learning", "Analytical tools development",
    "Data quality assurance", "Performance testing", "Infrastructure design and documentation",
    "Probability", "Visualization optimization", "Data source identification", "Database development and maintenance",
    "Data modeling", "Effective data visualization"
]

soft_skills_keywords = [
    "comunicación", "liderazgo", "trabajo en equipo", "resolución de problemas", "adaptabilidad",
    "creatividad", "pensamiento crítico", "gestión del tiempo", "atención al detalle", "orientación a resultados",
    "aprendizaje continuo", "manejo del estrés", "negociación", "persuasión", "empatía", "toma de decisiones",
    "passionate", "problem-solving", "intellectual curiosity", "eager to learn", "collaboration",
    "english communication", "teamwork", "initiative", "proactive", "entrepreneur mindset", "resilience"
]

hard_skills_keywords = [
    "Python", "Java", "SQL", "Excel", "Machine Learning", "Data Analysis", "Project Management", "Cloud Computing",
    "R", "C++", "Scala", "MATLAB", "Julia", "MongoDB", "Hadoop", "Spark", "Cassandra", "Redis",
    "Tableau", "Power BI", "Looker", "SAS", "D3.js", "TensorFlow", "PyTorch", "Keras", "Scikit-learn",
    "Natural Language Processing", "NLP", "Deep Learning", "AWS", "Azure", "Google Cloud Platform", "GCP",
    "Docker", "Kubernetes", "CI/CD", "Continuous Integration", "Continuous Deployment", "Análisis de regresión",
    "Series temporales", "Probabilidad", "Álgebra lineal", "Optimización"
]

# Construcción de expresiones regulares
technical_skills_regex = r'(?i)(?<!\w)(?:' + '|'.join(re.escape(skill) for skill in technical_skills_keywords) + r')(?!\w)'
hard_skills_regex = r'(?i)(?<!\w)(?:' + '|'.join(re.escape(skill) for skill in hard_skills_keywords) + r')(?!\w)'
soft_skills_regex = r'(?i)(?<!\w)(?:' + '|'.join(re.escape(skill) for skill in soft_skills_keywords) + r')(?!\w)'

# Loop through the list of job IDs and get each URL
for job_id in id_list:
    job_url = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    job_response = requests.get(job_url)
    print(job_response.status_code)
    job_soup = BeautifulSoup(job_response.text, "html.parser")

    job_post = {}
    job_post["job_link"] = f"https://www.linkedin.com/jobs/view/{job_id}"

    try:
        job_post["job_title"] = job_soup.find("h2", {"class":"top-card-layout__title"}).text.strip()
    except:
        job_post["job_title"] = None

    try:
        job_post["company_name"] = job_soup.find("a", {"class": "topcard__org-name-link"}).text.strip()
    except:
        job_post["company_name"] = None

    try:
        company_location_div = job_soup.find("div", {"class": "topcard__flavor-row"})
        job_post["location"] = company_location_div.find_all("span")[-1].text.strip()
    except Exception as e:
        print(f"Error obteniendo ubicación: {e}")
        job_post["location"] = "No especificado"

    job_description_section = job_soup.find("section", {"class": "description"})
    if job_description_section:
        job_post["job_description"] = job_description_section.get_text(" ").strip()
    else:
        job_post["job_description"] = None

    if job_post["job_description"]:
        job_requirements = job_post["job_description"]

        job_post["requires_english"] = "Inglés" in job_requirements or "English" in job_requirements

        keywords_degree = ["Licenciatura", "Ingeniería", "Maestría", "Doctorado", "PhD", "Bachillerato", "Recién egresados"]
        job_post["requires_degree"] = next((keyword for keyword in keywords_degree if keyword in job_requirements), "No especificado")

        english_levels = ["Inglés conversacional", "Inglés avanzado", "Inglés intermedio", "conversational English", "advanced English", "intermediate English"]
        job_post["english_level"] = next((level for level in english_levels if level in job_requirements), "No especificado")

        job_post["technical_skills"] = ", ".join(re.findall(technical_skills_regex, job_requirements, re.IGNORECASE))
        job_post["hard_skills"] = ", ".join(re.findall(hard_skills_regex, job_requirements, re.IGNORECASE))
        job_post["soft_skills"] = ", ".join(re.findall(soft_skills_regex, job_requirements, re.IGNORECASE))

    else:
        job_post["requires_english"] = False
        job_post["requires_degree"] = 'N/A'
        job_post["english_level"] = "No especificado"
        job_post["hard_skills"] = []
        job_post["technical_skills"] = []
        job_post["soft_skills"] = []

    try:
        salary_text = job_soup.find("span", {"class": "jobs-unified-top-card__salary"}).text.strip()
        salary_values = salary_text.replace("Base pay range", "").strip().split(" - ")
        job_post["salary_min"] = salary_values[0].strip()
        job_post["salary_max"] = salary_values[1].strip() if len(salary_values) > 1 else salary_values[0].strip()
    except:
        job_post["salary_min"] = "No especificado"
        job_post["salary_max"] = "No especificado"

    job_list.append(job_post)

200
200
200
200
200
200
200
200
200
200


In [19]:
#   Check if the list contains all the desired data
job_list

[{'job_link': 'https://www.linkedin.com/jobs/view/4128247651',
  'job_title': 'Analista de datos Jr',
  'company_name': 'Walmart de México y Centroamérica',
  'location': 'Miguel Hidalgo, Mexico City, Mexico',
  'job_description': 'Direct message the job poster from Walmart de México y Centroamérica \n \n \n \n              \n          \n        Danica Benitez\n\n \n \n \n \n \n   \n            \n        Danica Benitez\n\n \n \n \n              \n        Talent Advisor\n      \n             \n \n   \n \n \n \n \n \n \n \n \n \n \n ¿Qué harás? Prevenir la pérdida de activos y bienes para tiendas de autoservicio, a través de la generación y alineación de estrategias digitales, negociación y análisis de data para así alcanzar los objetivos del TOTAL LOSS definidos en la empresa, contribuyendo a la utilidad y resultado financiero de las unidades. Analizar información robusta que genere una toma de decisión eficiente, para así dar el seguimiento y accionables necesarios con la operación en 

In [20]:
# Create a pandas DataFrame using the list of job dictionaries 'job_list'
jobs_df = pd.DataFrame(job_list)

jobs_df['hard_skills'].to_string()
jobs_df['technical_skills'].to_string()
jobs_df['soft_skills'].to_string()

jobs_df

,job_link,job_title,company_name,location,job_description,requires_english,requires_degree,english_level,technical_skills,hard_skills,soft_skills,salary_min,salary_max
0,https://www.linkedin.com/jobs/view/4128247651,Analista de datos Jr,Walmart de México y Centroamérica,"Miguel Hidalgo, Mexico City, Mexico",Direct message the job poster from Walmart de ...,False,Ingeniería,No especificado,Python,"Excel, SQL, Python","negociación, toma de decisiones, comunicación,...",No especificado,No especificado
1,https://www.linkedin.com/jobs/view/4137031981,Analista de datos,Walmart de México y Centroamérica,"Tepotzotlán, México, Mexico",Direct message the job poster from Walmart de ...,True,Licenciatura,Inglés conversacional,,"Java, SQL, tableau","adaptabilidad, liderazgo, trabajo en equipo, L...",No especificado,No especificado
2,https://www.linkedin.com/jobs/view/4114591465,Analista de Datos,Omnicom Media Group Mexico,"Mexico City, Mexico","Acerca de nosotros Somos una agencia global, e...",True,No especificado,Inglés avanzado,Python,"Excel, Python, SQL, R, Power BI",,No especificado,No especificado
3,https://www.linkedin.com/jobs/view/4084883735,Data Analyst,Sezzle,Mexico,About Sezzle: With a mission to financially em...,True,No especificado,No especificado,"data analysis, statistics, data science, machi...","SQL, data analysis, machine learning, SQL, Pyt...","passionate, collaboration, initiative, problem...",No especificado,No especificado
4,https://www.linkedin.com/jobs/view/4126320994,Analista de datos,Softtek,Mexico,Direct message the job poster from Softtek \n ...,False,Ingeniería,No especificado,"AWS, data modeling","SQL, AWS",,No especificado,No especificado
5,https://www.linkedin.com/jobs/view/4122468515,Analista de datos BI,Empresa Confidencial,"Miguel Hidalgo, Mexico City, Mexico",Empresa lider en manufactura Solicita: Análisi...,True,No especificado,Inglés conversacional,,Excel,negociación,No especificado,No especificado
6,https://www.linkedin.com/jobs/view/4137904013,Data Analyst Jr,Aviva,"Mexico City, Mexico",Estamos buscando un Data Analyst que se integr...,False,No especificado,No especificado,"Python, Google Cloud","Tableau, Power BI, Looker, SQL, Python, R","toma de decisiones, atención al detalle",No especificado,No especificado
7,https://www.linkedin.com/jobs/view/4132496530,Analista de Datos para Riesgo y Cumplimiento -...,Mercado Libre,"Mexico City, Mexico",En Mercado Libre estamos democratizando el com...,False,No especificado,No especificado,Python,"SQL, Python, R",,No especificado,No especificado
8,https://www.linkedin.com/jobs/view/4119058631,Data Analyst,Navistar Financial - México,"Mexico City, Mexico",Direct message the job poster from Navistar Fi...,True,No especificado,Inglés intermedio,Python,"Excel, SQL, Power BI, Python",,No especificado,No especificado
9,https://www.linkedin.com/jobs/view/4079130909,Data Analyst,SEGULA Technologies,"Mexico City, Mexico",Company Description Expand your playground wit...,True,No especificado,No especificado,,"Power BI, Project Management",,No especificado,No especificado


In [21]:
# #Save data to CSV file
# jobs_df.to_csv('Linkedin_Scraper.csv', index = False, encoding='utf-8')

In [22]:
df = pd.DataFrame(job_list)
column_order = ["location", "company_name", "job_title", "requires_degree", "english_level", "technical_skills", "hard_skills", "soft_skills", "salary_min", "job_link"]
df = df.reindex(columns=column_order)

# Exportar el DataFrame a un archivo CSV con codificación UTF-8
df.to_csv('job_list.csv', index=False, encoding='utf-8')